# Linear Iterative Solvers

The aim is to solve the system:
$$
A \textbf{x} = \textbf{b}
$$
where $A \in \mathbb{R}^{m \times n}$ is a linear operator typically represented as a sparse matrix.

General reference:
* Saad, Y. *Iterative Methods for Sparse Linear Systems*, SIAM, 2003.
*  [TNL solvers](https://mmg-gitlab.fjfi.cvut.cz/doc/tnl/md_Tutorials_Solvers_Linear_tutorial_Linear_solvers.html)

In [ ]:
import torch

In [ ]:
!git clone https://github.com/grinisrit/noa.git

In [ ]:
!mkdir -p build

In [ ]:
noa_location = 'noa'
from torch.utils.cpp_extension import load

You need the following files:
* iterative-methods.hh
* iterative-methods.cc
* iterative-methods.cu

In [ ]:
iterative_methods = load(name='iterative_methods',
             build_directory='./build',
             sources=['iterative-methods.cc'],
             extra_include_paths=[f'{noa_location}/src', '.'],    
             extra_cflags=['-O3 -std=c++17 -fopenmp'],
             verbose=True)

In [ ]:
iterative_methods_cuda = load(name='iterative_methods_cuda',
             build_directory='./build',
             sources=['iterative-methods.cu'],
             extra_include_paths=[f'{noa_location}/src', '.'],    
             extra_cflags=['-O3 -std=c++17'],
             extra_cuda_cflags=['-std=c++17 --expt-relaxed-constexpr --expt-extended-lambda'],
             verbose=True)  if torch.cuda.is_available() else None

In [ ]:
def generate_tridiagonal(n, l, d, u):
    c = torch.tensor([-1,0,1]).repeat(n)
    r = torch.arange(n).repeat_interleave(3)
    cr = c + r
    rows = r[1:-1]
    cols = cr[1:-1]
    vals = torch.tensor([l, d ,u]).repeat(n)[1:-1]
    Ai = torch.stack([rows, cols])
    A = torch.sparse_coo_tensor(Ai, vals, (n,n))
    return A

In [ ]:
n = 100000
A = generate_tridiagonal(n, -0.5, 2.5, -1.5)

In [ ]:
Ad = A.to_dense()

In [ ]:
A_cu = A.cuda()

In [ ]:
Acsr = A.to_sparse_csr()

In [ ]:
Acsr_cu = A_cu.to_sparse_csr() # Acsr.cuda() is not supported as of torch 1.10.2

CSR format with indices data over `int` is preferred.

In [ ]:
def get_csr_data(csr_matrix):
    values =  csr_matrix.values() 
    crow_indices = csr_matrix.crow_indices().int()
    col_indices = csr_matrix.col_indices().int()
    return (crow_indices, col_indices, values)

In [ ]:
Acsr_data = get_csr_data(Acsr)

In [ ]:
Acsr_data_cu = get_csr_data(Acsr_cu)

In [ ]:
x0 = torch.ones(n)

In [ ]:
x0_cu = x0.cuda()

In [ ]:
b = Acsr @ x0
b[:5]

In [ ]:
b_cu = Acsr_cu @  x0_cu
b_cu[:5]

## Stationary methods

Those are fixed point methods. Numerically more robust, they might suffer from computational costs due to slow convergence.

### Jacobi method

* [Wiki](https://en.wikipedia.org/wiki/Jacobi_method) reference
* [TNL docs](https://mmg-gitlab.fjfi.cvut.cz/doc/tnl/classTNL_1_1Solvers_1_1Linear_1_1Jacobi.html)

In [ ]:
x = iterative_methods.jacobi_solve(*Acsr_data, b, 1.0)
torch.dist(x,x0)/n

In [ ]:
x_cu = iterative_methods_cuda.jacobi_solve(*Acsr_data_cu, b_cu, 1.0)
torch.dist(x_cu.cpu(), x0)/n

In [ ]:
%timeit iterative_methods.jacobi_solve(*Acsr_data, b, 1.0)

In [ ]:
%timeit iterative_methods_cuda.jacobi_solve(*Acsr_data_cu, b_cu, 1.0)

### SOR method

* [Wiki](https://en.wikipedia.org/wiki/Successive_over-relaxation) reference
* [TNL docs](https://mmg-gitlab.fjfi.cvut.cz/doc/tnl/classTNL_1_1Solvers_1_1Linear_1_1SOR.html)

In [ ]:
x = iterative_methods.sor_solve(*Acsr_data, b, 1.0)
torch.dist(x,x0)/n

In [ ]:
x_cu = iterative_methods_cuda.sor_solve(*Acsr_data_cu, b_cu, 1.0)
torch.dist(x_cu.cpu(), x0)/n

In [ ]:
%timeit iterative_methods.sor_solve(*Acsr_data, b, 1.0)

In [ ]:
%timeit iterative_methods_cuda.sor_solve(*Acsr_data_cu, b_cu, 1.0)